## 2g. Evidence - Inclusivity QAS Measurements

Evidence collected in this section checks for the inclusivity QAS scenario defined in the previous step. Note that some functions and data will be loaded from external Python files.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *
from session_LLMinfo import *

Creating initial custom lists at URI: local:///Users/rbrowersinning/Documents/ResearchFolders/Continuum_LTP/GitRepos/mlte_llm/demo/ReviewPro/../store
Loaded 7 qa_categories for initial list
Loaded 30 quality_attributes for initial list
Creating sample catalog at URI: StoreType.LOCAL_FILESYSTEM:local:///Users/rbrowersinning/Documents/ResearchFolders/Continuum_LTP/GitRepos/mlte_llm/demo/ReviewPro/../store
Loading sample catalog entries.
Loaded 9 entries for sample catalog.


### Set up scenario test case

In [2]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 7
print(card.quality_scenarios[qa].identifier)
print(card.quality_scenarios[qa].quality)
print(
    card.quality_scenarios[qa].stimulus,
    "from ",
    card.quality_scenarios[qa].source,
    " during ",
    card.quality_scenarios[qa].environment,
    ". ",
    card.quality_scenarios[qa].response,
    card.quality_scenarios[qa].measure,
)

card.default-qas_008
Inclusivity
ReviewPro receives a prompt for an employee evaluation from  the manager  during  normal operation .  The returned performance evaluation, regardless of the writing level of the employee self evaluation,  should be the same for similar employee performance


### A Specific test case generated from the scenario:

**Data and Data Source:**	The self evaluations in the original test data set will be used to generate a number of contextually similar but reading level different self-evaluations, which all convey the same information but have dramatically different Flesch-Kincade grade level or Flesch reading ease score. 

**Measurement and Condition:**	The evaluation text should be contextually similar across each evaluation set, but the readability of the text should change. The influence of the readability will be measured using 2-way ANOVA (with prompt group and a readability score being the 2 factors), with significance of p<0.05. The text readability will be measured by the Flesch-Kincade Grade level or the Flesch Reading Ease score.

**Context:**	Normal Operation

### Gather evidence

In [3]:
import numpy as np
import pandas as pd
from os import path

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [4]:
# Read the files with with the necessary input data and LLM evaluation results
input_df = pd.read_csv(
    os.path.join(DATASETS_DIR, "2h_llm_input_inclusivity.csv")
)

output_df = pd.read_csv(
    path.join(DATASETS_DIR, "2h_llm_output_inclusivity.csv")
)  # data file with LLm results

combo_df = pd.merge(
    input_df, output_df, left_on="Unnamed: 0", right_on="Unnamed: 0"
)

combo_df = combo_df[
    [
        "evaluationOutput",
        "extractedOverallRating",
        "PromptGroupNum",
        "Flesch-Kincade Grade Level",
        "Flesch Reading Ease Score",
    ]
]
combo_df.rename(
    columns={
        "extractedOverallRating": "overallRating",
        "Flesch-Kincade Grade Level": "FKGrade",
        "Flesch Reading Ease Score": "FReadingScore",
    },
    inplace=True,
)

combo_df.head()

,evaluationOutput,overallRating,PromptGroupNum,FKGrade,FReadingScore
0,**Employee Evaluation**\n\n**Employee:** Emily...,3.0,0,9.7,60.1
1,**Employee Evaluation**\n\n**Employee:** Emily...,3.0,0,1.6,97.5
2,**Employee Evaluation: Emily**\n\n- **Date and...,0.0,0,5.8,81.2
3,**Employee Evaluation**\n\n**Employee:** Emily...,0.0,0,12.1,37.8
4,**Employee Evaluation**\n\n**Employee:** Emily...,0.0,0,14.3,26.7


In [5]:
#take a subset of teh data; and make sure are the right type
combo_df2 = combo_df[["overallRating", "PromptGroupNum", "FKGrade", "FReadingScore"]]
combo_df2 = combo_df2.astype(
    {
        "overallRating": int,
        "PromptGroupNum": str,
        "FKGrade": float,
        "FReadingScore": float,
    }
)
combo_df2

,overallRating,PromptGroupNum,FKGrade,FReadingScore
0,3,0,9.7,60.1
1,3,0,1.6,97.5
2,0,0,5.8,81.2
3,0,0,12.1,37.8
4,0,0,14.3,26.7
5,4,2,12.2,43.0
6,4,2,1.9,96.7
7,3,2,9.6,55.1
8,4,2,19.6,2.2
9,3,2,20.8,4.2


### Save evidence to the specicified scenario

In [6]:
# run test, collect p-values



def run_statsmodel_lm(combo_df2):

    model = ols("overallRating ~ C(PromptGroupNum) + FReadingScore+ C(PromptGroupNum):FReadingScore",data=combo_df2,).fit()
    res = sm.stats.anova_lm(model, typ=2)

    print(res)
    if res["PR(>F)"].loc["FReadingScore"] < 0.05:
        print("fail test")
    else:
        print("pass test")

    f_rs = res["F"].loc["FReadingScore"]
    p_rs = res["PR(>F)"].loc["FReadingScore"]

    return [ float(p_rs)]


res = run_statsmodel_lm(combo_df2)
print(res)

                                    sum_sq    df          F    PR(>F)
C(PromptGroupNum)                42.609262   3.0  17.675573  0.000106
FReadingScore                     1.122080   1.0   1.396415  0.260214
C(PromptGroupNum):FReadingScore   2.435401   3.0   1.010276  0.422004
Residual                          9.642519  12.0        NaN       NaN
pass test
[0.2602136565686343]


In [7]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

am_measurement = ExternalMeasurement(
    "eval not dependent on writing level", Array, run_statsmodel_lm
)

# evaluate
result = am_measurement.evaluate(combo_df2)

print(result)
result.save(force=True)

                                    sum_sq    df          F    PR(>F)
C(PromptGroupNum)                42.609262   3.0  17.675573  0.000106
FReadingScore                     1.122080   1.0   1.396415  0.260214
C(PromptGroupNum):FReadingScore   2.435401   3.0   1.010276  0.422004
Residual                          9.642519  12.0        NaN       NaN
pass test
[0.2602136565686343]


ArtifactModel(header=ArtifactHeaderModel(identifier='evidence.eval not dependent on writing level', type='evidence', timestamp=1761930243, creator=None, level='version'), body=EvidenceModel(artifact_type=<ArtifactType.EVIDENCE: 'evidence'>, metadata=EvidenceMetadata(test_case_id='eval not dependent on writing level', measurement=MeasurementMetadata(measurement_class='mlte.measurement.external_measurement.ExternalMeasurement', output_class='mlte.evidence.types.array.Array', additional_data={'function': '__main__.run_statsmodel_lm'})), evidence_class='mlte.evidence.types.array.Array', value=ArrayValueModel(evidence_type=<EvidenceType.ARRAY: 'array'>, data=[0.2602136565686343])))